In [508]:
from gsmmutils import MyModel
import gsmmutils
gsmmutils.model.model_validator.DATA_PATH =  "../results"
import seaborn as sns
DATA_PATH = r"../data"
from os.path import join
sns.set(rc={'figure.figsize':(35,8.27)})
from cobra.flux_analysis import pfba

# Basic Simulations

1. Load the models

In [514]:
ds = MyModel(join(DATA_PATH, 'models/model_ds.xml'), 'e_Biomass__cytop')
ds.set_prism_reaction("PRISM_white_LED__extr")
print(ds.slim_optimize())
ng = MyModel(join(DATA_PATH, 'models/model_ng.xml'), 'e_Biomass__cytop')
ng.set_prism_reaction("PRISM_solar_litho__extr")
print(ng.slim_optimize())
pl = MyModel(join(DATA_PATH, 'models/model_pl.xml'), 'e_Biomass__cytop')
pl.set_prism_reaction("PRISM_fluorescent_warm_18W__extr")
pl.reactions.e_Biomass_ht__cytop.bounds = (0,0)
print(pl.slim_optimize())

Loading

Reactions: 4740
Metabolites: 4182
Genes: 1353
Model loaded
0.1422356064049512


2. Simple pFBA

In [6]:
sol = pfba(ds)
ds_original_co2 = sol.fluxes["EX_C00011__dra"]
ds.summary(sol)

Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,6.743,0,0.00%
C00009__extr,EX_C00009__dra,0.03278,0,0.00%
C00011__extr,EX_C00011__dra,8.21,1,99.70%
C00059__extr,EX_C00059__dra,0.02218,0,0.00%
C00080__extr,EX_C00080__dra,0.0558,0,0.00%
C00120__extr,EX_C00120__dra,0.0009328,10,0.11%
C00205__extr,EX_C00205__dra,235.4,0,0.00%
C00244__extr,EX_C00244__dra,0.5305,0,0.00%
C00305__extr,EX_C00305__dra,0.002294,0,0.00%
C00378__extr,EX_C00378__dra,0.0005361,12,0.08%


In [32]:
sol = pfba(ng)
ng_original_co2 = sol.fluxes["EX_C00011__dra"]
ng.summary(sol)

Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,1.161,0,0.00%
C00009__extr,EX_C00009__dra,0.005525,0,0.00%
C00011__extr,EX_C00011__dra,1.445,1,99.91%
C00059__extr,EX_C00059__dra,0.00445,0,0.00%
C00080__extr,EX_C00080__dra,0.009963,0,0.00%
C00120__extr,EX_C00120__dra,4.91E-05,10,0.03%
C00205__extr,EX_C00205__dra,42.87,0,0.00%
C00244__extr,EX_C00244__dra,0.133,0,0.00%
C00305__extr,EX_C00305__dra,0.0004984,0,0.00%
C00378__extr,EX_C00378__dra,2.822E-05,12,0.02%


In [510]:
sol = pfba(pl)
pl_original_co2 = sol.fluxes["EX_C00011__dra"]
pl.summary(sol)

Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,4.375,0,0.00%
C00009__extr,EX_C00009__dra,0.02206,0,0.00%
C00011__extr,EX_C00011__dra,6.587,1,99.91%
C00059__extr,EX_C00059__dra,0.02327,0,0.00%
C00080__extr,EX_C00080__dra,1.369,0,0.00%
C00120__extr,EX_C00120__dra,0.0002237,10,0.03%
C00205__extr,EX_C00205__dra,380,0,0.00%
C00244__extr,EX_C00244__dra,0.8956,0,0.00%
C00305__extr,EX_C00305__dra,0.002959,0,0.00%
C00378__extr,EX_C00378__dra,0.0001286,12,0.02%


3. Growth on different carbon sources

D. salina

In [17]:
carbon_sources = ["C00267", "C00033",  "C00116"]

with ds as model:
    print(f"Photoautotrophic: {model.slim_optimize()}")

for carbon_source in carbon_sources:
    with ds as model:
        model.reactions.NGAM__lum.bounds= (0, 1000)
        model.exchanges.EX_C00205__dra.lower_bound = 0
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = ds_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print("Heterotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())
        
for carbon_source in carbon_sources:
    with ds as model:
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = ds_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print(f"Mixotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())

Photoautotrophic: 0.20061690015141123
Heterotrophic with  alpha-D-Glucose 0.05336963258152157
Heterotrophic with  Acetate 0.005652815847295737
Heterotrophic with  Glycerol 0.07524072162844091
Mixotrophic with  alpha-D-Glucose 0.40123380030281586
Mixotrophic with  Acetate 0.4012338003028232
Mixotrophic with  Glycerol 0.4012338003028245


N. gaditana

In [18]:
carbon_sources = ["C00267", "C00469", "C00181",  "C02336", "C00089", "C00962", "C00116"]

with ng as model:
    print(f"Photoautotrophic: {model.slim_optimize()}")
    
with ng as model:
    model.reactions.NGAM__lum.bounds= (0, 1000)
    model.exchanges.EX_C00205__dra.lower_bound = 0
    model.exchanges.EX_C00244__dra.lower_bound = -1000
    model.exchanges.EX_C00011__dra.lower_bound = ng_original_co2
    for carbon_source in carbon_sources:
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = ng_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print("Heterotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())
      
with ng as model:
    model.reactions.NGAM__lum.bounds= (0, 1000)
    model.exchanges.EX_C00244__dra.lower_bound = -1000
    model.exchanges.EX_C00011__dra.lower_bound = ng_original_co2
    for carbon_source in carbon_sources:
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = ng_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print("Mixotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())

with ng as model:
    model.exchanges.EX_C00244__dra.lower_bound = -0.133
    print(f"Autotrophic with nitrate: {model.slim_optimize()}")

with ng as model:
    model.exchanges.EX_C00244__dra.lower_bound = 0
    model.exchanges.EX_C00088__dra.lower_bound = -0.133
    print(f"Autotrophic with nitrite: {model.slim_optimize()}")
    
with ng as model:
    model.exchanges.EX_C00244__dra.lower_bound = 0
    model.exchanges.EX_C00014__dra.lower_bound = -0.133
    print(f"Autotrophic with ammonia: {model.slim_optimize()}")
    
    
with ng as model:
    model.exchanges.EX_C00244__dra.lower_bound = 0
    model.exchanges.EX_C00086__dra.lower_bound = -0.133/2
    print(f"Autotrophic with urea: {model.slim_optimize()}")

Photoautotrophic: 0.0301824095942282
Heterotrophic with  alpha-D-Glucose 0.009138349980751626
Heterotrophic with  Ethanol 0.029381698019256247
Heterotrophic with  D-Xylose 0.04879480110151199
Heterotrophic with  beta-D-Fructose 0.06785745453124516
Heterotrophic with  Sucrose 0.08642144188601457
Heterotrophic with  beta-D-Galactose 0.10449292685146681
Heterotrophic with  Glycerol 0.12395834412959379
Mixotrophic with  alpha-D-Glucose 0.060364819188441415
Mixotrophic with  Ethanol 0.09054722878268186
Mixotrophic with  D-Xylose 0.12072963837691006
Mixotrophic with  beta-D-Fructose 0.15091204797113758
Mixotrophic with  Sucrose 0.18109445756536535
Mixotrophic with  beta-D-Galactose 0.2112768671595931
Mixotrophic with  Glycerol 0.2414592767538206
Autotrophic with nitrate: 0.030182409594227234
Autotrophic with nitrite: 0.030182409594227907
Autotrophic with ammonia: 0.03018240959422917
Autotrophic with urea: 0.030182409594227307


P. lutheri

In [585]:
carbon_sources = ["C00267",
                  "C00033",
                  "C00116", 
                  "C00089"]

with pl as model:
    print(f"Photoautotrophic: {model.slim_optimize()}")

# Mixotrophic
for carbon_source in carbon_sources:
    with pl as model:
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = pl_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print("Mixotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())

# Heterotrophic
for carbon_source in carbon_sources:
    with pl as model:
        model.exchanges.EX_C00205__dra.lower_bound = 0
        model.reactions.NGAM__lum.bounds= (0, 1000)
        model.reactions.ATPm__cytop.bounds= (0, 1000)
        pl.reactions.e_Biomass_ht__cytop.bounds = (0, 1000)
        model.objective = "e_Biomass_ht__cytop"
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = pl_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print("Heterotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())

Photoautotrophic: 0.14223560640495164
Mixotrophic with  alpha-D-Glucose 0.2459559413409167
Mixotrophic with  Acetate 0.22404421677639977
Mixotrophic with  Glycerol 0.2561246016287602
Mixotrophic with  Sucrose 0.24595594134091725
Heterotrophic with  alpha-D-Glucose 0.09093689893602007
Heterotrophic with  Acetate 0.06707383932724918
Heterotrophic with  Glycerol 0.10209495107571545
Heterotrophic with  Sucrose 0.09093689893602028
